In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amexda/test_data.parquet
/kaggle/input/amexda/add_event.parquet
/kaggle/input/amexda/685404e30cfdb_submission_template.csv
/kaggle/input/amexda/data_dictionary.csv
/kaggle/input/amexda/offer_metadata.parquet
/kaggle/input/amexda/add_trans.parquet
/kaggle/input/amexda/train_data.parquet


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set paths
DATA_PATH = Path('/kaggle/input/amexda')
OUTPUT_PATH = Path('/kaggle/working/eda_results')

# Create output directory
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

def save_plot(fig, filename):
    fig.savefig(OUTPUT_PATH / filename, bbox_inches='tight')
    plt.close(fig)

# 1. Load datasets with progress monitoring
print("Loading datasets...")
train = pd.read_parquet(DATA_PATH/'train_data.parquet', engine='pyarrow')
test = pd.read_parquet(DATA_PATH/'test_data.parquet', engine='pyarrow')
meta = pd.read_parquet(DATA_PATH/'offer_metadata.parquet', engine='pyarrow')
events = pd.read_parquet(DATA_PATH/'add_event.parquet', engine='pyarrow')
trans = pd.read_parquet(DATA_PATH/'add_trans.parquet', engine='pyarrow')

# 2. Basic dataset inspection
def dataset_summary(df, name):
    summary = pd.DataFrame({
        'Dataset': name,
        'Rows': [df.shape[0]],
        'Columns': [df.shape[1]],
        'Missing_%': [df.isna().mean().mean() * 100],
        'Size_MB': [df.memory_usage(deep=True).sum() / (1024**2)]
    })
    return summary

summary_list = [
    dataset_summary(train, 'train'),
    dataset_summary(test, 'test'),
    dataset_summary(meta, 'metadata'),
    dataset_summary(events, 'events'),
    dataset_summary(trans, 'transactions')
]

summary_df = pd.concat(summary_list)
print(summary_df)
summary_df.to_csv(OUTPUT_PATH / 'dataset_summary.csv', index=False)

# 3. Target analysis (train only)
plt.figure(figsize=(10, 6))
ax = train['y'].value_counts(normalize=True).plot.bar()
plt.title('Target Distribution (Click Rate)')
plt.ylabel('Proportion')
plt.xticks([0,1], ['No Click (0)', 'Click (1)'], rotation=0)
save_plot(plt.gcf(), 'target_distribution.png')
# Convert 'y' to numeric type before analysis
train['y'] = pd.to_numeric(train['y'], errors='coerce')

# Now you can safely calculate CTR
ctr = train['y'].mean()
print(f"Overall Click-Through Rate: {ctr:.4%}")

# 4. Missing value analysis
def plot_missing(df, name):
    missing = df.isna().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    
    plt.figure(figsize=(12, 8))
    missing[:30].plot.bar()  # Top 30 features with missing values
    plt.title(f'Missing Values - {name}')
    plt.ylabel('Missing Count')
    save_plot(plt.gcf(), f'missing_{name}.png')
    return missing

train_missing = plot_missing(train, 'Training Data')
test_missing = plot_missing(test, 'Test Data')

# 5. Temporal analysis (using id5 date column)
train['date'] = pd.to_datetime(train['id5'], errors='coerce')
test['date'] = pd.to_datetime(test['id5'], errors='coerce')

plt.figure(figsize=(12, 6))
train.groupby(train['date'].dt.date)['y'].mean().plot()
plt.title('Daily Click-Through Rate Over Time')
plt.ylabel('CTR')
save_plot(plt.gcf(), 'daily_ctr_trend.png')

# 6. Feature distributions (sampled for efficiency)
def plot_feature_distributions(df, name, sample_size=10000):
    sample = df.sample(min(sample_size, len(df)), random_state=42)
    
    # Numerical features
    num_cols = sample.select_dtypes(include=np.number).columns
    for col in num_cols[:5]:  # First 5 numerical features
        plt.figure()
        sns.histplot(sample[col], kde=True)
        plt.title(f'Distribution of {col}')
        save_plot(plt.gcf(), f'dist_{name}_{col}.png')
    
    # Categorical features
    cat_cols = sample.select_dtypes(include='object').columns
    for col in cat_cols[:3]:  # First 3 categorical features
        plt.figure()
        sample[col].value_counts().head(10).plot.bar()
        plt.title(f'Top 10 Values - {col}')
        save_plot(plt.gcf(), f'categorical_{name}_{col}.png')

plot_feature_distributions(train, 'train')
plot_feature_distributions(test, 'test')

# 7. Additional datasets analysis
# Offer metadata analysis
plt.figure(figsize=(10, 6))
meta['f376'].plot.hist(bins=30)  # Discount rate distribution
plt.title('Discount Rate Distribution')
save_plot(plt.gcf(), 'discount_distribution.png')

# Events data analysis
events['event_date'] = pd.to_datetime(events['id4'], errors='coerce')
events['event_type'] = events['id7'].notna().map({True: 'Click', False: 'Impression'})

plt.figure(figsize=(10, 6))
events['event_type'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('Event Type Distribution')
save_plot(plt.gcf(), 'event_type_distribution.png')

# 8. Save memory by downcasting
def downcast(df):
    for col in df.select_dtypes(include='integer'):
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include='float'):
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df

train = downcast(train)
test = downcast(test)

# 9. Generate report
with open(OUTPUT_PATH / 'eda_report.md', 'w') as f:
    f.write("# AMEX Competition EDA Report\n")
    f.write(f"**Overall CTR**: {ctr:.4%}\n\n")
    f.write("## Dataset Summary\n")
    f.write(summary_df.to_markdown(index=False) + "\n\n")
    f.write("## Key Findings\n")
    f.write("- Target variable shows significant class imbalance\n")
    f.write("- Temporal patterns observed in CTR\n")
    f.write("- Several features contain missing values (see visualizations)\n")
    f.write("## Next Steps\n")
    f.write("- Feature engineering for temporal patterns\n")
    f.write("- Imputation strategy for missing values\n")
    f.write("- Class imbalance handling in modeling")

print("EDA complete! Results saved to:", OUTPUT_PATH)


Loading datasets...
        Dataset      Rows  Columns  Missing_%       Size_MB
0         train    770164      372  23.838103  14712.150325
0          test    369301      371  20.503509   7228.309033
0      metadata      4164       12  24.151457      2.168164
0        events  21457473        5  19.628018   4846.647804
0  transactions   6339465        9   0.002286   2718.020789
Overall Click-Through Rate: 4.8108%


/usr/local/lib/python3.11/dist-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


EDA complete! Results saved to: /kaggle/working/eda_results


In [10]:
# STEP-3 ──────────────────────────────────────────────────────────────
#  CLEAN DATA (no chained assignment warnings)  –  AMEX Campus Challenge
#  Assumes Step-2 EDA already ran and dataset “amexda” is attached.

import pandas as pd, numpy as np
from pathlib import Path

DATA_DIR = Path('/kaggle/input/amexda')
WORK_DIR = Path('/kaggle/working')
WORK_DIR.mkdir(exist_ok=True, parents=True)

train = pd.read_parquet(DATA_DIR/'train_data.parquet', engine='pyarrow')
test  = pd.read_parquet(DATA_DIR/'test_data.parquet',  engine='pyarrow')

# ─────────────────────────────────────────────────────────────────────
# 1.  Make sure the target is numeric
train['y'] = pd.to_numeric(train['y'], errors='coerce')

# ─────────────────────────────────────────────────────────────────────
# 2.  Universal cleaning helper (NO chained assignment)
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()                                         # prevent surprises if caller passes a view
    
    # 2-A  ±∞ ➔ NaN  (numeric cols only)
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)
    
    # 2-B  build imputation maps once
    median_map = {c: df[c].median() 
                  for c in num_cols if df[c].isna().any()}
    
    cat_cols   = df.select_dtypes(exclude=[np.number]).columns
    mode_map   = {c: (df[c].mode().iloc[0] 
                      if not df[c].mode().empty else 'Unknown')
                  for c in cat_cols if df[c].isna().any()}
    
    # single pass fill (avoids chained-assignment warning)
    df.fillna(value={**median_map, **mode_map}, inplace=True)
    
    # 2-C  down-cast to save RAM
    for c in df.select_dtypes(include='float').columns:
        df[c] = pd.to_numeric(df[c], downcast='float')
    for c in df.select_dtypes(include='int').columns:
        df[c] = pd.to_numeric(df[c], downcast='integer')
    
    # 2-D  drop exact duplicates (keeps row count ≤ original)
    if df.duplicated().any():
        df = df.drop_duplicates()
    return df

# ─────────────────────────────────────────────────────────────────────
# 3.  Apply cleaner
train = clean_dataframe(train)
test  = clean_dataframe(test)

# ─────────────────────────────────────────────────────────────────────
# 4.  Final sanity checks
assert not train.isna().any().any(), 'NaNs remain in train'
assert not test.isna().any().any(),  'NaNs remain in test'
assert not np.isinf(train.select_dtypes(include=[np.number])).any().any()
assert not np.isinf(test.select_dtypes(include=[np.number])).any().any()

# ─────────────────────────────────────────────────────────────────────
# 5.  Persist cleaned data
train.to_parquet(WORK_DIR/'train_clean.parquet', index=False, engine='pyarrow')
test.to_parquet (WORK_DIR/'test_clean.parquet',  index=False, engine='pyarrow')

print('✅  Cleaning finished → no warnings, files saved in /kaggle/working')


✅  Cleaning finished → no warnings, files saved in /kaggle/working


In [7]:
import os, glob, pathlib, pprint
pprint.pprint(glob.glob('/kaggle/working/**/*.parquet', recursive=True))

['/kaggle/working/test_clean.parquet', '/kaggle/working/train_clean.parquet']


In [21]:
import pandas as pd, numpy as np, gc, time, sys
from pathlib import Path

RAW_DIR = Path('/kaggle/input/amexda')
CLEAN_DIR = Path('/kaggle/working')
WORK_DIR = Path('/kaggle/working')
OUT_TRAIN = WORK_DIR / 'fe_v1_train.parquet'
OUT_TEST  = WORK_DIR / 'fe_v1_test.parquet'

# Fast-path: skip if already exists
if OUT_TRAIN.exists() and OUT_TEST.exists():
    print('✅ fe_v1 already exists – skipping regeneration.')
    sys.exit(0)

# Load clean train/test
train = pd.read_parquet(CLEAN_DIR / 'train_clean.parquet')
test  = pd.read_parquet(CLEAN_DIR / 'test_clean.parquet')

# Force id2 and id3 to int64 in train/test
train['id2'] = train['id2'].astype('int64')
train['id3'] = train['id3'].astype('int64')
test['id2'] = test['id2'].astype('int64')
test['id3'] = test['id3'].astype('int64')

# Load events and force id2/id3 to int64
events_cols = ['id2', 'id3', 'id4', 'id7']
events = pd.read_parquet(RAW_DIR / 'add_event.parquet', columns=events_cols)
events['id2'] = pd.to_numeric(events['id2'], errors='coerce').astype('int64')
events['id3'] = pd.to_numeric(events['id3'], errors='coerce').astype('int64')

events['imp']   = 1
events['click'] = events['id7'].notna().astype('int8')

# Aggregations
agg = (events.groupby(['id2', 'id3'])
               .agg(imp_cnt=('imp', 'sum'),
                    click_cnt=('click', 'sum'))
               .reset_index())
agg['ctr_id2_id3'] = agg['click_cnt'] / agg['imp_cnt']

offer_agg = (agg.groupby('id3')
                 .agg(offer_imp_cnt=('imp_cnt', 'sum'),
                      offer_click_cnt=('click_cnt', 'sum'))
                 .reset_index())
offer_agg['offer_ctr'] = offer_agg['offer_click_cnt'] / offer_agg['offer_imp_cnt']

cust_agg = (agg.groupby('id2')
                .agg(cust_imp_cnt=('imp_cnt', 'sum'),
                     cust_click_cnt=('click_cnt', 'sum'))
                .reset_index())
cust_agg['cust_ctr'] = cust_agg['cust_click_cnt'] / cust_agg['cust_imp_cnt']

del events
gc.collect()

# Merge into train/test
def enrich(df):
    df = df.copy()
    df['id2'] = df['id2'].astype('int64')
    df['id3'] = df['id3'].astype('int64')
    df = (df.merge(agg,       on=['id2', 'id3'], how='left')
            .merge(offer_agg, on='id3',          how='left')
            .merge(cust_agg,  on='id2',          how='left'))
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].fillna(0).astype('float32')
    return df

train = enrich(train)
test  = enrich(test)

train.to_parquet(OUT_TRAIN, index=False)
test.to_parquet(OUT_TEST,  index=False)
print('🎉 fe_v1 saved →', OUT_TRAIN.name, ',', OUT_TEST.name)


🎉 fe_v1 saved → fe_v1_train.parquet , fe_v1_test.parquet
